### **0. Import the relevant packages**

In [ ]:
import os

# Setup projects
SIMPLE_RAG_PROJECT = "simple-rag"
os.environ["PHOENIX_PROJECT_NAME"] = SIMPLE_RAG_PROJECT

In [ ]:
import pandas as pd
import phoenix as px
from phoenix.trace.llama_index import OpenInferenceTraceCallbackHandler
import nest_asyncio
from datasets import load_dataset
from dotenv import load_dotenv

In [ ]:
from tqdm import tqdm

from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

nest_asyncio.apply()  # needed for concurrent evals in notebook environments
pd.set_option("display.max_colwidth", 1000)

### **1. Launch Phoenix**
You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the OpenInferenceTraceCallbackHandler. Phoenix supports LlamaIndex's one-click observability which will automatically instrument your LlamaIndex application! You can consult our integration guide for a more detailed explanation of how to instrument your LlamaIndex application.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run the LlamaIndex application).

In [ ]:
session = px.launch_app()
callback_handler = OpenInferenceTraceCallbackHandler()

In [ ]:
# We can view the above data in the UI
session.view()

### **2. Run Your Query Engine and View Your Traces in Phoenix**

We've compiled a list of commonly asked questions about Qdrant. Let's download the sample queries and take a look.

In [ ]:
## Loading the Eval dataset
qdrant_qa = load_dataset("atitaarora/qdrant_docs_qna_ragas", split="train")
qdrant_qa_question = qdrant_qa.select_columns(['question'])

In [ ]:
qdrant_qa_question['question'][:10]

### *This example uses a QdrantVectorStore and uses the previously generated collection to work fully connected with Qdrant but you can use whatever LlamaIndex application you like.*

In [ ]:
load_dotenv()

In [ ]:
COLLECTION_NAME = "qdrant_docs_arize_dense"

In [ ]:
##Uncomment to initialise qdrant client in memory
client = qdrant_client.QdrantClient(
   location=":memory:",
)

##Uncomment below to connect to Qdrant Cloud
# client = QdrantClient(
#     os.environ.get("QDRANT_URL"), 
#     api_key=os.environ.get("QDRANT_API_KEY"),
# )

## Uncomment below to connect to local Qdrant
#client = qdrant_client.QdrantClient("http://localhost:6333")

In [ ]:
client.get_collections()

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.core.indices.vector_store import VectorIndexRetriever

vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = load_index_from_storage(storage_context)

retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode=VectorStoreQueryMode.DEFAULT,
    similarity_top_k=5
)

In [ ]:
query_engine = index.as_query_engine()
for query in tqdm(qdrant_qa_question['question'][:10]):
    try:
      query_engine.query(query)
      #retriever.retrieve(query)
    except Exception as e:
      pass